In [12]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Layer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Rescaling , GlobalAveragePooling2D
from tensorflow.keras import layers, optimizers, callbacks
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.applications import EfficientNetV2B2
from sklearn.metrics import confusion_matrix, classification_report
import gradio as gr

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
!unzip /content/drive/MyDrive/ColabNotebooks/gb.zip

Archive:  /content/drive/MyDrive/ColabNotebooks/gb.zip
replace TrashType_Image_Dataset/cardboard/cardboard_001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace TrashType_Image_Dataset/cardboard/cardboard_002.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace TrashType_Image_Dataset/cardboard/cardboard_003.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace TrashType_Image_Dataset/cardboard/cardboard_004.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: TrashType_Image_Dataset/cardboard/cardboard_004.jpg  
replace TrashType_Image_Dataset/cardboard/cardboard_005.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: TrashType_Image_Dataset/cardboard/cardboard_005.jpg  
  inflating: TrashType_Image_Dataset/cardboard/cardboard_006.jpg  
  inflating: TrashType_Image_Dataset/cardboard/cardboard_007.jpg  
  inflating: TrashType_Image_Dataset/cardboard/cardboard_008.jpg  
  inflating: TrashType_Image_Dataset/cardboard/cardboard_009.jpg  
  inflating: TrashType_Image_Dataset

In [15]:
dataset_dir = "/content/TrashType_Image_Dataset"
image_size = (124, 124)
batch_size = 32
seed = 42


In [16]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=seed,
    shuffle = True,
    image_size=image_size,
    batch_size=batch_size
)

Found 2527 files belonging to 6 classes.
Using 2022 files for training.


In [17]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="validation",
    seed=seed,
    shuffle = True,
    image_size=image_size,
    batch_size=batch_size
)
val_class= val_ds.class_names

Found 2527 files belonging to 6 classes.
Using 505 files for validation.


In [19]:
# Get the total number of batches in the validation dataset
val_batches = tf.data.experimental.cardinality(val_ds)

# Split the validation dataset into two equal parts:
# First half becomes the test dataset
test_ds = val_ds.take(val_batches // 2)

# Second half remains as the validation dataset
val_dat = val_ds.skip(val_batches // 2)

# Optimize test dataset by caching and prefetching to improve performance
test_ds_eval = test_ds.cache().prefetch(tf.data.AUTOTUNE)

In [20]:
print(train_ds.class_names)
print(val_class)
print(len(train_ds.class_names))

['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
6
